# Set the environment

In [1]:
# set the envoiroment
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem

## load the model

In [2]:
model = cobra.io.load_json_model("../models/updated_model.json")# Import the model.
model.solver = 'glpk'
model.reactions.T_flux_oxygen.bounds = (0,0) # block the import of oxygen
model.reactions.NGAM.bounds = (0,1000)
map_loc = "../data/fermentation.json"

## make the bounds of the small model

In [3]:
model1= model.copy()
## add the reaction for the maintainance of NADPH
reaction = Reaction('NADPH_recycle')
reaction.name = 'NADPH_recycle'
reaction.lower_bound = -100.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model1.add_reactions([reaction])
reaction.add_metabolites({'NADPH': -1.0,
                         'NADP+': 1.0,
                         "H+":1.0})
reaction.reaction
## add the reaction for the maintainance of NADH
reaction = Reaction('NADH_recycle')
reaction.name = 'NADH_recycle'
reaction.lower_bound = -100.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model1.add_reactions([reaction])
reaction.add_metabolites({'NADH': -1.0,
                         'NAD+': 1.0,
                         "H+":1.0})
reaction.reaction

total_reactions_id = [r.id for r in model1.reactions]
small_pathway_reactions = ["carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD","carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NADP","carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN",
        "carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN",'T_flux_H+','T_trans_H+','T_flux_H2O','T_trans_H2O','T_flux_phosphate','T_trans_phosphate',
        'carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN','carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN',
        'carb_enter_2.7.1.40_PEPDEPHOS__RXN','T_flux_D__glyceraldehyde__3__phosphate','NADPH_recycle','NADH_recycle']

for r in total_reactions_id:
    if r not in small_pathway_reactions:
        model1.reactions.get_by_id(r).upper_bound = 0
        model1.reactions.get_by_id(r).lower_bound = 0
    else:
        model1.reactions.get_by_id(r).bounds = (-10, 10)
model1.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (1,1)
model1.reactions.get_by_id("carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN").bounds = (0,1)


## when n <1, for example n=0,  no solution

In [4]:
M = model1.copy()
M.reactions.get_by_id('T_trans_phosphate').bounds = (-10,10)
reaction = Reaction('from_pyruvate_to_GAP')
reaction.name = 'from_pyruvate_to_GAP'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 10.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"pyruvate": -1.0,
                         "NAD+": -1.0,
                         "ATP": -0,
                         "D__glyceraldehyde__3__phosphate": 1.0,
                         "NADH": 1.0,
                          "H+": 1.0,
                         "ADP": 0})

M.objective = M.reactions.get_by_id('from_pyruvate_to_GAP')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
for rxn in l:
    print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn])

print ("GAPN flux is " + str(sol.fluxes.loc['carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN']),", GAPDH flux is " + str(-sol.fluxes.loc['carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN']))

b = show_map(sol,map_loc)
b    


T_flux_H+ H+_ex <=>  3.999999999999999
T_trans_H+ H+ <=> H+_ex 4.0
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.0
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.0
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate --> ATP + pyruvate 1.0
NADPH_recycle NADPH <=> H+ + NADP+ 0.9999999999999999
NADH_recycle NADH <=> H+ + NAD+ 1.0000000000000002
from_pyruvate_to_GAP NAD+ + pyruvate --> D__glyceraldehyde__3__phosphate + H+ + NADH 1.0000000000000002
GAPN flux is 1.0 , GAPDH flux is -0.0


cobra\util\solver.py:408 UserWarning: solver status is 'infeasible'


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when n <1, for example n=0.5, no feasible solution

In [5]:
M = model1.copy()
M.reactions.get_by_id('T_trans_phosphate').bounds = (-10,10)


reaction = Reaction('from_pyruvate_to_GAP')
reaction.name = 'from_pyruvate_to_GAP'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 10.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"pyruvate": -1.0,
                         "NAD+": -1.0,
                         "ATP": -0.5,
                         "D__glyceraldehyde__3__phosphate": 1.0,
                         "NADH": 1.0,
                          "H+": 1.0,
                         "ADP": 0.5})

M.objective = M.reactions.get_by_id('from_pyruvate_to_GAP')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
# for rxn in l:
#     print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn])
print ("GAPN flux is " + str(sol.fluxes.loc['carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN']),", GAPDH flux is " + str(-sol.fluxes.loc['carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN']))

b = show_map(sol,map_loc)
b    


GAPN flux is 1.0 , GAPDH flux is -0.0


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when n =1, GAPN flux 1, GAPDH flux 0

In [6]:
M = model1.copy()
M.reactions.get_by_id('T_trans_phosphate').bounds = (-10,10)


reaction = Reaction('from_pyruvate_to_GAP')
reaction.name = 'from_pyruvate_to_GAP'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 10.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"pyruvate": -1.0,
                         "NAD+": -1.0,
                         "ATP": -1,
                         "D__glyceraldehyde__3__phosphate": 1.0,
                         "NADH": 1.0,
                          "H+": 1.0,
                         "ADP": 1})

M.objective = M.reactions.get_by_id('from_pyruvate_to_GAP')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
# for rxn in l:
#     print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn])
print ("GAPN flux is " + str(sol.fluxes.loc['carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN']),", GAPDH flux is " + str(-sol.fluxes.loc['carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN']))

b = show_map(sol,map_loc)
b    


GAPN flux is 1.0 , GAPDH flux is -0.0


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when n =1.5, GAPN flux 0.5, GAPDH flux 0.5

In [7]:
M = model1.copy()
M.reactions.get_by_id('T_trans_phosphate').bounds = (-10,10)


reaction = Reaction('from_pyruvate_to_GAP')
reaction.name = 'from_pyruvate_to_GAP'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 10.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"pyruvate": -1.0,
                         "NAD+": -1.0,
                         "ATP": -1.5,
                         "D__glyceraldehyde__3__phosphate": 1.0,
                         "NADH": 1.0,
                          "H+": 1.0,
                         "ADP": 1.5})

M.objective = M.reactions.get_by_id('from_pyruvate_to_GAP')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("GAPN flux is " + str(sol.fluxes.loc['carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN']),", GAPDH flux is " + str(-sol.fluxes.loc['carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN']))

b = show_map(sol,map_loc)
b    


GAPN flux is 0.5 , GAPDH flux is 0.5


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when n =2, GAPN flux 0, GAPDH flux 1

In [8]:
M = model1.copy()
M.reactions.get_by_id('T_trans_phosphate').bounds = (-10,10)


reaction = Reaction('from_pyruvate_to_GAP')
reaction.name = 'from_pyruvate_to_GAP'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 10.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"pyruvate": -1.0,
                         "NAD+": -1.0,
                         "ATP": -2,
                         "D__glyceraldehyde__3__phosphate": 1.0,
                         "NADH": 1.0,
                          "H+": 1.0,
                         "ADP": 2})

M.objective = M.reactions.get_by_id('from_pyruvate_to_GAP')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("GAPN flux is " + str(sol.fluxes.loc['carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN']),", GAPDH flux is " + str(-sol.fluxes.loc['carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN']))

b = show_map(sol,map_loc)
b    


GAPN flux is 0.0 , GAPDH flux is 1.0


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when n =3, no solution 

In [9]:
M = model1.copy()
M.reactions.get_by_id('T_trans_phosphate').bounds = (-10,10)


reaction = Reaction('from_pyruvate_to_GAP')
reaction.name = 'from_pyruvate_to_GAP'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 10.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"pyruvate": -1.0,
                         "NAD+": -1.0,
                         "ATP": -3,
                         "D__glyceraldehyde__3__phosphate": 1.0,
                         "NADH": 1.0,
                          "H+": 1.0,
                         "ADP": 3})

M.objective = M.reactions.get_by_id('from_pyruvate_to_GAP')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("GAPN flux is " + str(sol.fluxes.loc['carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN']),", GAPDH flux is " + str(-sol.fluxes.loc['carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN']))
b = show_map(sol,map_loc)
b 

GAPN flux is 0.0 , GAPDH flux is 0.49999999999999994


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

# when adding ATPase reaction

In [10]:
M= model.copy()

reaction = Reaction('T_flux_pyruvate')
reaction.name = 'T_flux_pyruvate'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({'pyruvate': -1.0})
reaction.reaction

reaction = Reaction('T_flux_D__glyceraldehyde__3__phosphate')
reaction.name = 'T_flux_D__glyceraldehyde__3__phosphate'
reaction.lower_bound = -1  # This is the default
reaction.upper_bound = 0  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({
                         "D__glyceraldehyde__3__phosphate": -1.0})
reaction.reaction
## add the reaction for the maintainance of NADPH
reaction = Reaction('NADPH_recycle')
reaction.name = 'NADPH_recycle'
reaction.lower_bound = -100.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({'NADPH': -1.0,
                         'NADP+': 1.0,
                         "H+":1.0})
reaction.reaction
## add the reaction for the maintainance of NADH
reaction = Reaction('NADH_recycle')
reaction.name = 'NADH_recycle'
reaction.lower_bound = -100.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({'NADH': -1.0,
                         'NAD+': 1.0,
                         "H+":1.0})
reaction.reaction
total_reactions_id = [r.id for r in M.reactions]
small_pathway_reactions = ["carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD","carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NADP","carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN",
        "carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN",'T_flux_H+','T_trans_H+','T_flux_H2O','T_trans_H2O','T_flux_phosphate','T_trans_phosphate',
        'carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN','carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN',
        'carb_enter_2.7.1.40_PEPDEPHOS__RXN','T_flux_pyruvate','T_flux_D__glyceraldehyde__3__phosphate','NADPH_recycle','NADH_recycle']

for r in total_reactions_id:
    if r not in small_pathway_reactions:
        M.reactions.get_by_id(r).upper_bound = 0
        M.reactions.get_by_id(r).lower_bound = 0
    else:
        M.reactions.get_by_id(r).bounds = (-10, 10)
# m = [r for r in l if r not in list]
# for r in m:
#     M.reactions.get_by_id(r).upper_bound = 0
#     M.reactions.get_by_id(r).lower_bound = 0
#     else:
#         M.reactions.get_by_id(r).bounds = (-10, 10)
M.reactions.get_by_id("carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN").bounds = (0,1)
# M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (-1,1)

In [11]:
def different_ATPase_bound (a,b):
    M.reactions.get_by_id('NGAM').bounds =(a,a)
    M.reactions.get_by_id('T_flux_D__glyceraldehyde__3__phosphate').bounds =(-b,0)
    M.objective = M.reactions.get_by_id('T_flux_pyruvate')
    sol = M.optimize()
    l = sol.fluxes[abs(sol.fluxes)>1e-6].index
    for rxn in l:
        print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn],M.reactions.get_by_id(rxn).bounds)
    print ("GAPN flux is " + str(sol.fluxes.loc['carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN']),", GAPDH flux is " + str(-sol.fluxes.loc['carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN']))
    print (", ATPase flux is " + str(sol.fluxes.loc['NGAM']))
    b = show_map(sol,map_loc)
    return(b)

In [12]:
different_ATPase_bound(0.25,1)

NGAM ATP + H2O --> ADP + H+ + phosphate 0.25 (0.25, 0.25)
T_flux_H+ H+_ex <=>  1.4999999999999998 (-10, 10)
T_trans_H+ H+ <=> H+_ex 1.5 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.6249999999999998 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.6249999999999998 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 0.625 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 0.625 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 0.625 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP 0.37500000000000006 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phosph

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 0

In [13]:
different_ATPase_bound(0,1)

T_flux_H+ H+_ex <=>  0.9999999999999998 (-10, 10)
T_trans_H+ H+ <=> H+_ex 0.9999999999999999 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.5000000000000001 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.5000000000000001 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 0.49999999999999994 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 0.4999999999999999 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 0.4999999999999999 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP 0.5 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phosphate 0.5000000000000

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 0.25

## when ATPase bounds is set at 0.5

In [14]:
different_ATPase_bound(0.5,1)

NGAM ATP + H2O --> ADP + H+ + phosphate 0.5 (0.5, 0.5)
T_flux_H+ H+_ex <=>  1.9999999999999996 (-10, 10)
T_trans_H+ H+ <=> H+_ex 1.9999999999999998 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.75 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.75 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 0.7499999999999999 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 0.75 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 0.75 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP 0.25000000000000017 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phosphate 0

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 1

In [15]:
different_ATPase_bound(1,1)

NGAM ATP + H2O --> ADP + H+ + phosphate 1.0 (1, 1)
T_flux_H+ H+_ex <=>  2.9999999999999996 (-10, 10)
T_trans_H+ H+ <=> H+_ex 3.0 (-10, 10)
T_flux_H2O H2O_ex <=>  -1.0 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -1.0 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.0 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.0 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.0 (-10, 10)
T_flux_phosphate phosphate_ex <=>  1.0 (-10, 10)
T_trans_phosphate phosphate_ex <=> phosphate -0.9999999999999999 (-10, 10)
T_flux_pyruvate pyruvate <=>  1.0 (-10, 10)
T_flux_D__glyceraldehyde__3__phosphate D__glyceraldehyde__3__phosphate <--  -1.0 (-1, 0)
NADPH_recycle NADPH <=> H+ + NADP+ 1.0 (-10, 10)
GAPN flux is 1.0 

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 1.25

In [16]:
different_ATPase_bound(1.25,1)

NGAM ATP + H2O --> ADP + H+ + phosphate 1.25 (1.25, 1.25)
T_flux_H+ H+_ex <=>  3.2500000000000004 (-10, 10)
T_trans_H+ H+ <=> H+_ex 3.2500000000000004 (-10, 10)
T_flux_H2O H2O_ex <=>  -1.0000000000000004 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -1.0000000000000004 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 0.7500000000000003 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.0 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.0 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.0 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP -0.24999999999999972 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 1.5

In [17]:
different_ATPase_bound(1.5,1)

NGAM ATP + H2O --> ADP + H+ + phosphate 1.5 (1.5, 1.5)
T_flux_H+ H+_ex <=>  3.5 (-10, 10)
T_trans_H+ H+ <=> H+_ex 3.5 (-10, 10)
T_flux_H2O H2O_ex <=>  -1.0 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -1.0 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 0.5 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.0 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.0 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.0 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP -0.49999999999999994 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phosphate -0.5 (-10, 10)
T_flux_phosphate phosphate_ex <=> 

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 2

In [18]:
different_ATPase_bound(2,1)

NGAM ATP + H2O --> ADP + H+ + phosphate 2.0 (2, 2)
T_flux_H+ H+_ex <=>  4.0 (-10, 10)
T_trans_H+ H+ <=> H+_ex 4.0 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.9999999999999996 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.9999999999999996 (-10, 10)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.0 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.0 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.0 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP -1.0000000000000002 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phosphate -1.0000000000000004 (-10, 10)
T_flux_phosphate phosphate_ex <=>  0.9999999999999996 (-10, 10)
T_trans_phosphate phosphate_ex <=> phosphate -0.9999999999999994 (-10

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 2.5

In [19]:
different_ATPase_bound(2.5,1)

NGAM ATP + H2O --> ADP + H+ + phosphate 2.5 (2.5, 2.5)
T_flux_H+ H+_ex <=>  4.500000000000001 (-10, 10)
T_trans_H+ H+ <=> H+_ex 4.500000000000001 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.9999999999999994 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.9999999999999994 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH -0.5 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.0000000000000004 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.0000000000000004 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.0000000000000004 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP -1.5000000000000002 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

# when adding ATPase reaction,T_flux_D__glyceraldehyde__3__phosphate bound was set to (-1.5,1.5)

## when ATPase bounds is set at 0

In [20]:
different_ATPase_bound(0,1.5)

T_flux_H+ H+_ex <=>  0.9999999999999998 (-10, 10)
T_trans_H+ H+ <=> H+_ex 0.9999999999999999 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.5 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.5 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 0.5 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 0.5 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 0.5 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP 0.49999999999999994 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phosphate 0.5 (-10, 10)
T_flux_phosphate phosphate_ex <=>  0.5 (-10, 10)
T_trans_phos

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 0.25

In [21]:
different_ATPase_bound(0.25,1.5)

NGAM ATP + H2O --> ADP + H+ + phosphate 0.25 (0.25, 0.25)
T_flux_H+ H+_ex <=>  1.4999999999999998 (-10, 10)
T_trans_H+ H+ <=> H+_ex 1.5 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.6249999999999998 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.6249999999999998 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 0.625 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 0.625 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 0.625 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP 0.3750000000000001 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phospha

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 0.5

In [22]:
different_ATPase_bound(0.5,1.5)

NGAM ATP + H2O --> ADP + H+ + phosphate 0.5 (0.5, 0.5)
T_flux_H+ H+_ex <=>  1.9999999999999996 (-10, 10)
T_trans_H+ H+ <=> H+_ex 1.9999999999999998 (-10, 10)
T_flux_H2O H2O_ex <=>  -0.75 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -0.75 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 0.7499999999999999 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 0.75 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 0.75 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP 0.25000000000000006 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+ + phosphate 0

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 1

In [23]:
different_ATPase_bound(1,1.5)

NGAM ATP + H2O --> ADP + H+ + phosphate 1.0 (1, 1)
T_flux_H+ H+_ex <=>  2.9999999999999996 (-10, 10)
T_trans_H+ H+ <=> H+_ex 3.0 (-10, 10)
T_flux_H2O H2O_ex <=>  -1.0 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -1.0 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.0 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.0 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.0 (-10, 10)
T_flux_phosphate phosphate_ex <=>  1.0 (-10, 10)
T_trans_phosphate phosphate_ex <=> phosphate -0.9999999999999999 (-10, 10)
T_flux_pyruvate pyruvate <=>  1.0 (-10, 10)
T_flux_D__glyceraldehyde__3__phosphate D__glyceraldehyde__3__phosphate <--  -1.0 (-1.5, 0)
NADPH_recycle NADPH <=> H+ + NADP+ 1.0 (-10, 10)
GAPN flux is 1.

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 1.25

In [24]:
different_ATPase_bound(1.25,1.5)

NGAM ATP + H2O --> ADP + H+ + phosphate 1.25 (1.25, 1.25)
T_flux_H+ H+_ex <=>  3.5 (-10, 10)
T_trans_H+ H+ <=> H+_ex 3.5 (-10, 10)
T_flux_H2O H2O_ex <=>  -1.1249999999999996 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -1.1249999999999996 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.1250000000000002 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.1250000000000002 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.1250000000000002 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP -0.12500000000000003 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 1.5

In [25]:
different_ATPase_bound(1.5,1.5)

NGAM ATP + H2O --> ADP + H+ + phosphate 1.5 (1.5, 1.5)
T_flux_H+ H+_ex <=>  3.999999999999999 (-10, 10)
T_trans_H+ H+ <=> H+_ex 3.9999999999999996 (-10, 10)
T_flux_H2O H2O_ex <=>  -1.2499999999999996 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -1.2499999999999996 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.25 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.25 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.25 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP -0.24999999999999983 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH <=> D__glyceraldehyde__3__phosphate + NAD+

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 1.75

In [26]:
different_ATPase_bound(1.75,1.5)

NGAM ATP + H2O --> ADP + H+ + phosphate 1.75 (1.75, 1.75)
T_flux_H+ H+_ex <=>  4.500000000000001 (-10, 10)
T_trans_H+ H+ <=> H+_ex 4.500000000000001 (-10, 10)
T_flux_H2O H2O_ex <=>  -1.3749999999999993 (-10, 10)
T_trans_H2O H2O <=> H2O_ex -1.3749999999999993 (-10, 10)
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN D__glyceraldehyde__3__phosphate + H2O + NADP+ --> 3__phospho__D__glycerate + H+ + NADPH 1.0 (0, 1)
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN 3__phospho__D__glycerate <=> 2__phospho__D__glycerate 1.3750000000000004 (-10, 10)
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN 2__phospho__D__glycerate <=> H2O + phosphoenolpyruvate 1.3750000000000004 (-10, 10)
carb_enter_2.7.1.40_PEPDEPHOS__RXN ADP + phosphoenolpyruvate <=> ATP + pyruvate 1.3750000000000004 (-10, 10)
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN 3__phospho__D__glycerate + ATP <=> 1,3__bisphospho__D__glycerate + ADP -0.3750000000000002 (-10, 10)
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD 1,3__bisphospho__D__glycerate + H+ + NADH

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## when ATPase bounds is set at 2

In [ ]:
different_ATPase_bound(2,1.5)

## when ATPase bounds is set at 2.25

In [ ]:
different_ATPase_bound(2.25,1.5)

## when ATPase bounds is set at 2.5

In [ ]:
different_ATPase_bound(2.5,1.5)

## when ATPase bounds is set at 2.75

In [ ]:
different_ATPase_bound(2.75,1.5)

## when ATPase bounds is set at 3

In [ ]:
different_ATPase_bound(3,1.5)